In [11]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import shutil
import random
import os

In [12]:
full_set_plader = "Data/KD hele plader"
train_set_plader = "Data/KD train plader"
test_set_plader = "Data/KD Test plader"
train_set_tiles = "Data/KD train tiles" 

train_set_tiles_mixed = train_set_tiles + "/blandet"

test_size=0.2

crop_percentage = 0
circle_size = 0.0

In [13]:
def copy_files(files, destination):
    for f in files:
        shutil.copy(os.path.join(full_set_plader, f), os.path.join(destination, f))

In [14]:
# Sikre, at destinationsmapperne eksisterer
os.makedirs(train_set_plader, exist_ok=True)
os.makedirs(train_set_plader, exist_ok=True)

# Hent alle filnavne fra data-mappen
all_files = [f for f in os.listdir(full_set_plader) if os.path.isfile(os.path.join(full_set_plader, f))]

# Bland listen i state 42 og lav et tilfældigt split
random.Random(42).shuffle(all_files)
split_point = int((1 - test_size) * len(all_files))

# Fordel filerne i train og test mapper
train_files = all_files[:split_point]
test_files = all_files[split_point:]

# Kopier filerne
copy_files(train_files, train_set_plader)
copy_files(test_files, test_set_plader)

Laver tile split

In [15]:
def get_tiles(image):
    # laver en tom liste 
    tiles = []
    # kører et for loop hvor elementer vil blive tilføjet til listen, hvor y repræsenterer en række af billedet
    for y in range(5):
        tiles.append([])
    # kører et nested loop, hvor billedet bliver delt op i en tavel med 25 kvadrater af 100,100 px og tilføjer til listen tiles
        for x in range(5):
            tiles[-1].append(image[y*100:(y+1)*100, x*100:(x+1)*100])
    return tiles

In [16]:
def zoom_tile(tile):
    height, width = tile.shape[:2]
    
    # Beregner hvor meget der skal beskæres fra hver side
    crop_height = int(height * crop_percentage)
    crop_width = int(width * crop_percentage)
    
    # Opdaterer udsnittets start- og slutpunkter for at fjerne de yderste 5%
    cropped_tile = tile[crop_height:height-crop_height, crop_width:width-crop_width]
    
    return cropped_tile

In [17]:
def remove_circle_from_tile(tile):
    if tile.shape[2] == 3:
        tile = cv.cvtColor(tile, cv.COLOR_BGR2BGRA)
    
    height, width = tile.shape[:2]
    diameter = int(circle_size * min(width, height))
    center = (width // 2, height // 2)
    
    mask = np.zeros((height, width), dtype=np.uint8)
    cv.circle(mask, center, diameter // 2, (255), -1)
    
    # Fjerner cirklen fra alfakanalen, så den bliver gennemsigtig
    tile[mask == 255, 3] = 0

    return tile

In [19]:
all_tiles = []

for filename in os.listdir(train_set_plader):
    image_path = os.path.join(train_set_plader, filename)

    if not os.path.isfile(image_path):
        print(f"Image not found: {image_path}")
        continue
    
    image = cv.imread(image_path)
    tiles = get_tiles(image)
    
    for y, row in enumerate(tiles):
        for x, tile in enumerate(row):
            tile = zoom_tile(tile)
            tile = remove_circle_from_tile(tile)
            all_tiles.append(tile)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
all_tiles